# Project Part-1
## Colab Notebook For CNN
**Connect Four**

Author: Pranath Reddy Kumbam

UFID: 8512-0977

- Notebook for training a simple CNN model on Balanced Connect Four Dataset

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive

In [ ]:
cd My \Drive

In [ ]:
cd DLCG/Project1

In [ ]:
# GPU Info
!nvidia-smi 

In [ ]:
# Get CUDA Info
!nvcc --version

In [ ]:
# Import libraries 
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import pickle
from tqdm.notebook import tqdm

# Define CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.classifier = nn.Sequential(
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.Conv2d(6, 12, 3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(72, 32),
            nn.ReLU(),
            nn.Linear(32, 3)
        )

    def forward(self, x):
        x = self.classifier(x)
        return x

# Push the model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

# Load Data
data = pd.read_csv('./Data/connectfour.data', sep= ',', header = None)
encoding = {"x":2, "o":1, "b":0, "win":0, "loss":1, "draw":2}
data = data.replace(encoding).to_numpy()

# Balance the Data based on Class with lowest No Of samples
data_win = np.asarray([sample for sample in data if sample[-1] == 1])
data_loss = np.asarray([sample for sample in data if sample[-1] == 0])
data_draw = np.asarray([sample for sample in data if sample[-1] == 2])
data = (np.concatenate((data_win[:data_draw.shape[0]], data_loss[:data_draw.shape[0]], data_draw)))
np.random.shuffle(data)

# Split the data into validation set
x, x_val, y, y_val = train_test_split(data[:,:-1], data[:,-1].reshape(-1), test_size=0.05)

# Reshape samples into 2D
x = x.reshape(x.shape[0], 1, 6, 7)
x_val = x_val.reshape(x_val.shape[0], 1, 6, 7)
print(x.shape)
print(x_val.shape)

# Shuffle
x, y = shuffle(x, y, random_state=0)
x_val, y_val = shuffle(x_val, y_val, random_state=0)

# MLP Optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-5)
criteria = nn.CrossEntropyLoss() # Loss Function
n_epochs = 200 # Training Epochs

# Multi-Class Classification
# Use K-Fold Cross Validation
print("Multi-Class Classification\n")
MLP_Scores = []
fold_index = 1
for train_index, test_index in KFold(n_splits=10).split(x):
    print('Fold: ' + str(fold_index) + '\n')
    x_tr, x_ts = x[train_index], x[test_index]
    y_tr, y_ts = y[train_index], y[test_index]

    model.train()
    for epoch in tqdm(range(1, n_epochs+1)):
      for i in range(int(x_tr.shape[0]/100)):

        data = torch.from_numpy(x_tr[(100*i): (100*i)+100].astype(np.float32))
        if torch.cuda.is_available():
          data = data.cuda()
        labels = torch.tensor(y_tr[(100*i): (100*i)+100], dtype=torch.long, device=device)
        optimizer.zero_grad()
        outputs = model(data)
        _, preds = torch.max(model(data).data, 1)
        correct = (preds == labels).float().sum()
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    x_ts_torch = torch.from_numpy(x_ts.astype(np.float32)).cuda()
    _, yp_CNN = torch.max(model(x_ts_torch).data, 1)
    yp_CNN = yp_CNN.cpu().detach().numpy()

    acc_CNN = accuracy_score(y_ts, yp_CNN)
    print("CNN Accuracy: " + str(acc_CNN) + '\n')
    MLP_Scores.append(acc_CNN)
    print("CNN Confusion Matrix: " + '\n')
    confmat = confusion_matrix(y_ts, yp_CNN, normalize='true')
    for row in confmat:
        print(*row, sep="\t")
    print("")
    print("________________________________________________________  \n")

    # Reset Model and optimizer
    for layer in model.children():
      if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-5)

    fold_index += 1

print("Results: ")
print("CNN Accuracy: " + str(np.mean(MLP_Scores)) + " +/- " + str(np.std(MLP_Scores)))

# Train to deploy model
model.train()
for epoch in tqdm(range(1, n_epochs+1)):
    for i in range(int(x.shape[0]/100)):

        data = torch.from_numpy(x[(100*i): (100*i)+100].astype(np.float32))
        if torch.cuda.is_available():
          data = data.cuda()
        labels = torch.tensor(y[(100*i): (100*i)+100], dtype=torch.long, device=device)
        optimizer.zero_grad()
        outputs = model(data)
        _, preds = torch.max(model(data).data, 1)
        correct = (preds == labels).float().sum()
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()

# Export trained model
torch.save(model, './CNN_Connect4_Balanced.pth')